In [ ]:
!pip install scalene

In [ ]:
import sys
print(sys.version)

3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]


In [ ]:
%load_ext scalene

The scalene extension is already loaded. To reload it, use:
  %reload_ext scalene


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim

import math

In [ ]:
def torchtest():
    dtype = torch.float
    #device = torch.device("cpu")
    device = torch.device("cuda:0")  # Uncomment this to run on GPU
    # device = torch.device("cuda")  # Uncomment this to run on GPU

    # Create Tensors to hold input and outputs.
    # By default, requires_grad=False, which indicates that we do not need to
    # compute gradients with respect to these Tensors during the backward pass.
    # x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
    q = torch.linspace(-math.pi, math.pi, 5000000, device=device, dtype=dtype)
    x = torch.linspace(-math.pi, math.pi, 5000000, device=device, dtype=dtype)
    y = torch.sin(x)

    # Create random Tensors for weights. For a third order polynomial, we need
    # 4 weights: y = a + b x + c x^2 + d x^3
    # Setting requires_grad=True indicates that we want to compute gradients with
    # respect to these Tensors during the backward pass.
    a = torch.randn((), device=device, dtype=dtype, requires_grad=True)
    b = torch.randn((), device=device, dtype=dtype, requires_grad=True)
    c = torch.randn((), device=device, dtype=dtype, requires_grad=True)
    d = torch.randn((), device=device, dtype=dtype, requires_grad=True)

    learning_rate = 1e-6
    for t in range(2000):
        # Forward pass: compute predicted y using operations on Tensors.
        y_pred = a + b * x + c * x ** 2 + d * x ** 3

        # Compute and print loss using operations on Tensors.
        # Now loss is a Tensor of shape (1,)
        # loss.item() gets the scalar value held in the loss.
        #     loss = (y_pred - y).pow(2).sum()
        loss = (y_pred - y).sum()
        if t % 100 == 99:
            print(t, loss.item())

        # Use autograd to compute the backward pass. This call will compute the
        # gradient of loss with respect to all Tensors with requires_grad=True.
        # After this call a.grad, b.grad. c.grad and d.grad will be Tensors holding
        # the gradient of the loss with respect to a, b, c, d respectively.
        loss.backward()

        # Manually update weights using gradient descent. Wrap in torch.no_grad()
        # because weights have requires_grad=True, but we don't need to track this
        # in autograd.
        with torch.no_grad():
            a -= learning_rate * a.grad
            b -= learning_rate * b.grad
            c -= learning_rate * c.grad
            d -= learning_rate * d.grad

            # Manually zero the gradients after updating weights
            a.grad = None
            b.grad = None
            c.grad = None
            d.grad = None

    print(f'Result: y = {a.item()} + {b.item()} x + {c.item()} x^2 + {d.item()} x^3')

In [ ]:
%scrun --version

SCRUN MAGIC
Scalene version 1.5.33 (2024.01.02)
Scalene failed to initialize.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/scalene/scalene_profiler.py", line 1877, in run_profiler
    Scalene.process_args(args)
  File "/usr/local/lib/python3.10/dist-packages/scalene/scalene_profiler.py", line 1797, in process_args
    time.perf_counter() + Scalene.__args.profile_interval
AttributeError: 'Namespace' object has no attribute 'profile_interval'



StopJupyterExecution: 

In [ ]:
%scrun torchtest()

SCRUN MAGIC
99 -29278793728.0
199 -58837041152.0
299 -88395128832.0
399 -117953019904.0
499 -147510919168.0
599 -177068802048.0
699 -206626684928.0
799 -236184584192.0
899 -265742483456.0
999 -295300399104.0
1099 -324858281984.0
1199 -354416164864.0
1299 -383974080512.0
1399 -413531963392.0
1499 -443089879040.0
1599 -472647761920.0
1699 -502205644800.0
1799 -531763560448.0
1899 -561321476096.0
1999 -590879326208.0
Result: y = -9999.580078125 + 1.3090906143188477 x + -32899.6015625 x^2 + -0.2016664296388626 x^3
                          /content/_ipython-input-10-profile: % of time = 100.00% (5.278s) out of 5.278s.                          
       ╷       ╷       ╷       ╷       ╷                                                                                           
       │Time   │–––––– │–––––– │–––––– │                                                                                           
  Line │Python │native │system │GPU    │/content/_ipython-input-10-profile              

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [ ]:
%scrun --help

usage: scalene [-h] [--version] [--column-width COLUMN_WIDTH] [--outfile OUTFILE] [--html]
               [--json] [--cli] [--web] [--viewer] [--reduced-profile]
               [--profile-interval PROFILE_INTERVAL] [--cpu] [--cpu-only] [--gpu] [--memory]
               [--profile-all] [--profile-only PROFILE_ONLY] [--profile-exclude PROFILE_EXCLUDE]
               [--use-virtual-time] [--cpu-percent-threshold CPU_PERCENT_THRESHOLD]
               [--cpu-sampling-rate CPU_SAMPLING_RATE]
               [--allocation-sampling-window ALLOCATION_SAMPLING_WINDOW]
               [--malloc-threshold MALLOC_THRESHOLD] [--program-path PROGRAM_PATH]
               [--memory-leak-detector] [--on | --off]

Scalene: a high-precision CPU and memory profiler, version 1.5.19 (2023.01.06)
]8;id=235435;https://github.com/plasma-umass/scalene\https://github.com/plasma-umass/scalene]8;;\

command-line:
  % scalene [options] your_program.py [--- --your_program_args] 
or
  % python3 -m scalene [options] your_program.py [--- --your_program_args] 

in Jupyter, line mode:
  %scrun [options] statement

in Jupyter, cell mode:
  %%scalene [options]
   your code here


options:
  -h, --help            show this help message and exit
  --version             prints the version number for this release of Scalene and exits
  --column-width COLUMN_WIDTH
                        Column width for profile output (default: 132)
  --outfile OUTFILE     file to hold profiler output (default: stdout)
  --html                output as HTML (default: web)
  --json                output as JSON (default: web)
  --cli                 forces use of the command-line
  --web                 opens a web tab to view the profile (saved as 'profile.html')
  --viewer              only opens the web UI (https://plasma-umass.org/scalene-gui/)
  --reduced-profile     generate a reduced profile, with non-zero lines only (default: False)
  --profile-interval PROFILE_INTERVAL
                        output profiles every so many seconds (default: inf)
  --cpu                 profile CPU time (default:  True )
  --cpu-only            profile CPU time (deprecated: use --cpu )
  --gpu                 profile GPU time and memory (default: True )
  --memory              profile memory (default: True )
  --profile-all         profile all executed code, not just the target program (default: only the target program)
  --profile-only PROFILE_ONLY
                        profile only code in filenames that contain the given strings, separated by commas 
(default: no restrictions)
  --profile-exclude PROFILE_EXCLUDE
                        do not profile code in filenames that contain the given strings, separated by commas 
(default: no restrictions)
  --use-virtual-time    measure only CPU time, not time spent in I/O or blocking (default: False)
  --cpu-percent-threshold CPU_PERCENT_THRESHOLD
                        only report profiles with at least this percent of CPU time (default: 1%)
  --cpu-sampling-rate CPU_SAMPLING_RATE
                        CPU sampling rate (default: every 0.01s)
  --allocation-sampling-window ALLOCATION_SAMPLING_WINDOW
                        Allocation sampling window size, in bytes (default: 10485767 bytes)
  --malloc-threshold MALLOC_THRESHOLD
                        only report profiles with at least this many allocations (default: 100)
  --program-path PROGRAM_PATH
                        The directory containing the code to profile (default: the path to the profiled program)
  --memory-leak-detector
                        EXPERIMENTAL: report likely memory leaks (default: True)
  --on                  start with profiling on (default)
  --off                 start with profiling off

When running Scalene in the background, you can suspend/resume profiling
for the process ID that Scalene reports. For example:

   % python3 -m scalene  yourprogram.py &
 Scalene now profiling process 12345
   to suspend profiling: python3 -m scalene.profile --off --pid 12345


StopJupyterExecution: 

In [ ]:
%scrun --version

Scalene version 1.5.20 (2023.02.27)


StopJupyterExecution: 

In [ ]:
import numpy as np

In [ ]:
%%scalene

for i in range(1000):
  b = np.array(range(1000))
  a = np.array(np.random.uniform(0, 100, size=10000))

Scalene: An exception of type SyntaxError occurred. Arguments:
('invalid syntax', ('<unknown>', 1, 1, '%%scalene\n', 1, 2))
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/scalene/scalene_profiler.py", line 2000, in run_profiler
    exit_status = profiler.profile_code(
  File "/usr/local/lib/python3.10/dist-packages/scalene/scalene_profiler.py", line 1810, in profile_code
    did_output = Scalene.output_profile()
  File "/usr/local/lib/python3.10/dist-packages/scalene/scalene_profiler.py", line 909, in output_profile
    json_output = Scalene.__json.output_profiles(
  File "/usr/local/lib/python3.10/dist-packages/scalene/scalene_json.py", line 380, in output_profiles
    enclosing_regions = ScaleneAnalysis.find_regions(code_str)
  File "/usr/local/lib/python3.10/dist-packages/scalene/scalene_analysis.py", line 45, in find_regions
    tree = ast.parse(src)
  File "/usr/lib/python3.10/ast.py", line 50, in parse
    return compile(source, filename, mode,

In [ ]:
%%scalene --web --profile-all

# Train an image classifier on CIFAR

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# device = 'cpu'
print(device)

# Load & normalize CIFAR

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=0)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=0)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

print('Complete loading CIFAR')

# Define a CNN

net = Net().to(device)
print(next(net.parameters()).device)

# Define a loss fun & optimizer

criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

print('Complete creating model, criterion & optimizer')

# Train the net

for epoch in range(2):  # loop over the dataset multiple times

    print(f'epoch {epoch}')
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # print(f'i = {i}')
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            # print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

# Test the net

correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

cuda:0
Files already downloaded and verified
Files already downloaded and verified
Complete loading CIFAR
cuda:0
Complete creating model, criterion & optimizer
epoch 0
epoch 1
Finished Training
Accuracy of the network on the 10000 test images: 54 %
Scalene: An exception of type SyntaxError occurred. Arguments:
('invalid syntax', ('<unknown>', 1, 1, '%%scalene --web --profile-all\n', 1, 2))
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/scalene/scalene_profiler.py", line 2000, in run_profiler
    exit_status = profiler.profile_code(
  File "/usr/local/lib/python3.10/dist-packages/scalene/scalene_profiler.py", line 1810, in profile_code
    did_output = Scalene.output_profile()
  File "/usr/local/lib/python3.10/dist-packages/scalene/scalene_profiler.py", line 909, in output_profile
    json_output = Scalene.__json.output_profiles(
  File "/usr/local/lib/python3.10/dist-packages/scalene/scalene_json.py", line 380, in output_profiles
    enclosing_region